# Operations on word vectors

Because word embeddings are very computationally expensive to train, most ML practitioners load a pre-trained set of embeddings. 

Tasks performed below: 
- Load pre-trained word vectors, and measure similarity using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______. 
- Modify word embeddings to reduce their gender bias 

In [1]:
import numpy as np
from w2v_utils import *

Using TensorFlow backend.


#### Load the word vectors
* A 50-dimensional GloVe vectors to represent words. 
- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.

#### Embedding vectors versus one-hot vectors
* Since one-hot vectors do not do a good job of capturing the level of similarity between words (every one-hot vector has the same Euclidean distance from any other one-hot vector).
* Embedding vectors such as GloVe vectors provide much more useful information about the meaning of individual words. 
* Can see this by measuring the similarity between two words. 

In [2]:
words, word_to_vec_map = read_glove_vecs('../../readonly/glove.6B.50d.txt')

# 1 - Cosine similarity

To measure the similarity between two words, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

* $u \cdot v$ is the dot product (or inner product) of two vectors
* $||u||_2$ is the norm (or length) of the vector $u$
* $\theta$ is the angle between $u$ and $v$. 
* The cosine similarity depends on the angle between $u$ and $v$. 
    * If $u$ and $v$ are very similar, their cosine similarity will be close to 1.
    * If they are dissimilar, the cosine similarity will take a smaller value. 

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center> **Figure 1**: The cosine of the angle between two vectors is a measure their similarity</center></caption>

In [3]:
# FUNCTION: cosine_similarity

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # Computed the dot product between u and v
    dot = np.dot(u,v)
    # Computed the L2 norm of u
    norm_u = np.sqrt(np.sum(u**2))
    
    # Computed the L2 norm of v
    norm_v = np.sqrt(np.sum(v**2))
    # Computed the cosine similarity defined by formula (1)
    cosine_similarity = dot/(norm_u*norm_v)
    
    return cosine_similarity

In [4]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
son = word_to_vec_map["son"]
daughter = word_to_vec_map["daughter"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]
woman = word_to_vec_map["woman"]
man = word_to_vec_map["man"]
#data_scientist = word_to_vec_map["data scientist"]
ceo = word_to_vec_map["ceo"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(father, son) = ", cosine_similarity(father, son))
print("cosine_similarity(father, daughter) = ", cosine_similarity(father, daughter))
print("cosine_similarity(mother, daughter) = ", cosine_similarity(mother, daughter))
print("cosine_similarity(mother, son) = ", cosine_similarity(mother, son))
print("cosine_similarity(son, daughter) = ", cosine_similarity(son, daughter))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))
print("cosine_similarity(woman, ceo) = ", cosine_similarity(woman, ceo))
print("cosine_similarity(man, ceo) = ", cosine_similarity(man, ceo))

cosine_similarity(father, mother) =  0.890903844289
cosine_similarity(father, son) =  0.952898319753
cosine_similarity(father, daughter) =  0.8904722279
cosine_similarity(mother, daughter) =  0.945622681322
cosine_similarity(mother, son) =  0.835079832624
cosine_similarity(son, daughter) =  0.896181617687
cosine_similarity(ball, crocodile) =  0.274392462614
cosine_similarity(france - paris, rome - italy) =  -0.675147930817
cosine_similarity(woman, ceo) =  0.110018071385
cosine_similarity(man, ceo) =  0.253214028564


## 2 - Word analogy task

* In the word analogy task, we complete the sentence:  
    <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. 

* An example is:  
    <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. 

* Aim is to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner:   
    $e_b - e_a \approx e_d - e_c$
* Will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 

In [5]:
# FUNCTION: complete_analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # converted words to lowercase
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    # Got the word embeddings e_a, e_b and e_c
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialized max_cosine_sim to a large negative number
    best_word = None                   # Initialized best_word with None, it will help keep track of the word to output

    # to avoid best_word being one of the input words, skipped the input words
    # placed the input words in a set for faster searching than a list
    # will re-use this set of input words inside the for-loop
    input_words_set = set([word_a, word_b, word_c])
    
    # looped over the whole word vector set
    for w in words:        
        if w in input_words_set:
            continue
        
        # Computed cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[w] - e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        
    return best_word

In [6]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large'),
                ('good', 'better', 'smart'), ('orange', 'juice', 'apple'), ('woman', 'ceo', 'man'), ('man', 'doctor', 'woman')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger
good -> better :: smart -> computers
orange -> juice :: apple -> juices
woman -> ceo :: man -> executive
man -> doctor :: woman -> nurse


## 3 - Debiasing word vectors

Here, we examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. 
Lets first see how the GloVe word embeddings relate to gender. We will first compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". 

We also compute g_avg - a more accurate representation considering $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$, etc. and averaging over them. 

In [7]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

g2 = word_to_vec_map['mother'] - word_to_vec_map['father']
g3 = word_to_vec_map['grandmother'] - word_to_vec_map['grandfather']
g4 = word_to_vec_map['girl'] - word_to_vec_map['boy']
g5 = word_to_vec_map['male'] - word_to_vec_map['female']
g_avg = (g+g2+g3+g4+g5)/5
print(g_avg)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]
[ 0.148392    0.244202   -0.340558   -0.037184   -0.039396    0.480372
  0.16131384  0.181874    0.38687572 -0.08054912 -0.043554   -0.554512
  0.4805256   0.01338     0.18141     0.111477   -0.45399    -0.1702714
  0.4254876   0.021674    0.085984    0.48978     0.1964664   0.3378936
  0.13083     0.14394    -0.143916    0.33009516  0.098226   -0.3573356
 -0.095262    0.2820958   0.0532714   0.05629157 -0.083

Now, we will consider the cosine similarity of different words with $g$. 

In [8]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.23163356146
marie 0.315597935396
sophie 0.318687898594
ronaldo -0.312447968503
priya 0.17632041839
rahul -0.169154710392
danielle 0.243932992163
reza -0.079304296722
katy 0.283106865957
yasmin 0.233138577679


As you can see, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not surprising, and the result seems acceptable. 

But let's try with some other words.

In [9]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.276919162564
guns -0.18884855679
science -0.0608290654093
arts 0.00818931238588
literature 0.0647250443346
warrior -0.209201646411
doctor 0.118952894109
tree -0.0708939917548
receptionist 0.330779417506
technology -0.131937324476
fashion 0.0356389462577
teacher 0.179209234318
engineer -0.0803928049452
pilot 0.00107644989919
computer -0.103303588739
singer 0.185005181365


Do you notice anything surprising? It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, "computer" is closer to "man" while "literature" is closer to "woman"! 

We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. 

### 3.1 - Neutralize bias for non-gender specific words 

The figure visualizes what neutralizing does. When using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which will be called $g_{\perp}$. In linear algebra, we say that the 49 dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49 dimensional, given the limitations of what we can draw on a 2D screen, we illustrate it using a 1 dimensional axis below. 

<img src="images/neutral.png" style="width:800px;height:300px;">
<caption><center> **Figure 2**: The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

Implemented `neutralize()` to remove the bias of words such as "receptionist" or "scientist". 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

In [10]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    # Selected word vector representation of "word" using word_to_vec_map
    e = word_to_vec_map[word]
    
    # Computed e_biascomponent using the formula given above
    e_biascomponent = np.dot(e,g)*g/(np.sqrt(np.sum(g**2))**2)
 
    # Neutralized e by subtracting e_biascomponent from it 
    e_debiased = e-e_biascomponent
    
    return e_debiased

In [11]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))


e_debiased = neutralize("receptionist", g_avg, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g_avg))

cosine similarity between receptionist and g, before neutralizing:  0.330779417506
cosine similarity between receptionist and g, after neutralizing:  5.84103233224e-18
cosine similarity between receptionist and g, after neutralizing:  4.34403894252e-17


### 3.2 - Equalization algorithm for gender-specific words

Equalization is applied to pairs of words that should differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. In pictures, this is how equalization works: 

<img src="images/equalize10.png" style="width:800px;height:400px;">


The derivation of the linear algebra is inspired by Bolukbasi et al., 2016. The key equations are: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$

In [20]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debiases gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    # Step 1: Selected word vector representation of "word" using word_to_vec_map.
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    # Step 2: Computed the mean of e_w1 and e_w2
    mu = (e_w1 + e_w2)/2

    # Step 3: Computed the projections of mu over the bias axis and the orthogonal axis
    mu_B = np.dot(mu,bias_axis)*bias_axis/(np.sqrt(np.sum(bias_axis**2))**2)
    mu_orth = mu-mu_B

    # Step 4: Using equations (7) and (8), computed e_w1B and e_w2B
    e_w1B = np.dot(e_w1,bias_axis)*bias_axis/(np.sqrt(np.sum(bias_axis**2))**2)
    e_w2B = np.dot(e_w2,bias_axis)*bias_axis/(np.sqrt(np.sum(bias_axis**2))**2)
        
    # Step 5: Adjusted the Bias part of e_w1B and e_w2B using the formulas (9) and (10)
    corrected_e_w1B = np.sqrt(np.abs(1-(np.sum(mu_orth**2))))*(e_w1B-mu_B)/np.abs((e_w1-mu_orth)-mu_B)
    corrected_e_w2B = np.sqrt(np.abs(1-(np.sum(mu_orth**2))))*(e_w2B-mu_B)/np.abs((e_w2-mu_orth)-mu_B)

    # Step 6: Debiased by equalizing e1 and e2 to the sum of their corrected projections
    e1 = corrected_e_w1B+mu_orth
    e2 = corrected_e_w2B+mu_orth
                                                                    
    return e1, e2

In [22]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g_avg))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g_avg))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g_avg))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g_avg))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.005020598481
cosine_similarity(word_to_vec_map["woman"], gender) =  0.388172975397

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.560766141401
cosine_similarity(e2, gender) =  0.606138026928


**References**:
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)
